In [2]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
from pytz import timezone
import math
import matplotlib.pyplot as plt

In [3]:
reddit_df = pd.read_csv('../data/reddit_text_sentiment.csv')
stocks_df = pd.read_csv("../data/stock_prices.csv")
companies_df = pd.read_csv("../data/companies.csv")
brands_df = pd.read_csv("../data/brands.csv")
industries_df = pd.read_csv("../data/industries.csv")

In [4]:
#helper function to convert utc to EST date
eastern = timezone('US/Eastern')
def utc_to_est(utc):
    return datetime.fromtimestamp(utc, tz = eastern)

In [5]:
#create new stocks column with datetime format of daily market close times
stocks_df["date"] = pd.to_datetime(stocks_df["date"]).values.astype(np.int64) // 10**6
stocks_df["date"] = (stocks_df["date"] + 57600000)//1000
stocks_df['datetime'] = stocks_df['date'].apply(utc_to_est)

In [6]:
#created new reddit column with datetime format of daily market close times
reddit_df['datetime'] = reddit_df['created_utc'].apply(utc_to_est)

In [7]:
#select relevant columns
reddit_df = reddit_df[['subreddit', 'datetime', 'score', 'compound', 'positive', 'neutral', 'negative']]

#add column that helps in counting posts when grouped
reddit_df['num_posts'] = 1
reddit_df['positive_posts'] = reddit_df['compound'] > 0.3
reddit_df['negative_posts'] = reddit_df['compound'] < -0.3

#replace zeros with insubstatial float in reddit scores
reddit_df.score = reddit_df.score.apply(lambda x: max(x, 0.01))

#weighted scores
for s in ['compound', 'negative', 'positive', 'neutral']:
    reddit_df['weighted_{0}'.format(s)] = reddit_df[s]*reddit_df['score'].apply(lambda x: math.log(x+1))
    
#add company_id as column
reddit_df = reddit_df.merge(brands_df[['subreddit', 'company_id']], on='subreddit')

In [8]:
#helper function that takes a dataframe and returns separate dataframes for each company
def separate_reddit(reddit_df, by='all'):
    if by not in ['all', 'company', 'industry']:
        print('argument invalid: must be = <all>, <company>, or <industry>')
        pass
    else:
        if by == 'industry':
            temp = reddit_df.merge(companies_df[['id', 'industry_id']], left_on = 'company_id', right_on='id')
            return [temp[temp['industry_id']==i][['industry_id', 'datetime','weighted_compound', 'weighted_negative', 'weighted_positive', 'weighted_neutral', 'num_posts', 'positive_posts', 'negative_posts']] for i in temp.industry_id.unique()]
        elif by == 'company':
            return [reddit_df[reddit_df['company_id']==i][['company_id', 'datetime','weighted_compound', 'weighted_negative', 'weighted_positive', 'weighted_neutral', 'num_posts', 'positive_posts', 'negative_posts']] for i in reddit_df.company_id.unique()]
        else:
            return [reddit_df[['company_id', 'datetime','weighted_compound', 'weighted_negative', 'weighted_positive', 'weighted_neutral', 'num_posts', 'positive_posts', 'negative_posts']]]

In [9]:
#groups data for a single company using sliding window - number of days specified in call
def group_data(df, by='all', days = 1):
    if by not in ['all', 'company', 'industry']:
        print('invalid arg: must be in [all, company, industry]')
        pass
    else:
        if by=='industry':
            i_id = df.industry_id.unique()[0] #save industry_id
            temp_df = df[['datetime', 'weighted_compound', 'weighted_positive', 'weighted_neutral', 'weighted_negative', 'num_posts', 'positive_posts', 'negative_posts']].groupby(pd.Grouper(key='datetime', freq='24h', base=11, label='right')).sum() #groupby day
            min_date = min(temp_df.index)
            max_date = max(temp_df.index)
            date_idx = [i for i in pd.date_range(min_date, max_date)] #new index
            temp_df = temp_df.reindex(date_idx).fillna(0).rolling(days).sum()[days-1:] #make dataframe with rolling window sum
            temp_df['industry_id'] = i_id #restore industry_id
            temp_df.reset_index(inplace=True)
            return temp_df
        elif by=='company':
            c_id = df.company_id.unique()[0] #save company_id
            temp_df = df[['datetime', 'weighted_compound', 'weighted_positive', 'weighted_neutral', 'weighted_negative', 'num_posts', 'positive_posts', 'negative_posts']].groupby(pd.Grouper(key='datetime', freq='24h', base=11, label='right')).sum() #groupby day
            min_date = min(temp_df.index)
            max_date = max(temp_df.index)
            date_idx = [i for i in pd.date_range(min_date, max_date)] #new index
            temp_df = temp_df.reindex(date_idx).fillna(0).rolling(days).sum()[days-1:] #make dataframe with rolling window sum
            temp_df['company_id'] = c_id #restore company_id
            temp_df.reset_index(inplace=True)
            return temp_df
        else:
            return pd.concat([group_data(d, 'company', days) for d in separate_reddit(df, 'company')])
    
    

In [10]:
def make_dataframes(df, separation='all', lookback=1, min_date = None, max_date = None):
    if min_date:
        df = df[df['datetime'] >= pd.to_datetime(min_date).tz_localize('US/Eastern')]
    if max_date:
        df = df[df['datetime'] <= pd.to_datetime(max_date).tz_localize('US/Eastern')]
    dfs = []
    separated = separate_reddit(df, separation)
    for d in separated:
        temp = group_data(d, separation, lookback)
        for i in ['weighted_compound', 'weighted_negative', 'weighted_positive', 'weighted_neutral', 'positive_posts', 'negative_posts']: #iterate over scores columns
            temp['avg_{}'.format(i)] = temp[i]/temp['num_posts'].apply(lambda x: max(x, 1)) #set avgerage weighted scores columns
        temp = temp.merge(stocks_df[['company_id', 'datetime', 'change_percent']], on=['company_id', 'datetime']) #add change_percent column
        dfs.append(temp)
    return dfs
        

In [11]:
from sklearn.linear_model import LinearRegression, LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [12]:
#helper function that finds optimal C for a given training set (logistic regression)
def cross_validate_logreg(X_train, y_train, c_s = [0.1, 0.3, 1, 3, 10, 30, 100]):
    c_s = [0.1, 0.3, 1, 3, 10, 30, 100]
    lg_cv = LogisticRegressionCV(Cs = c_s, scoring='f1').fit(X_train, y_train)
    return c_s[np.argmax(np.mean(lg_cv.scores_[True],axis=0))], np.max(np.mean(lg_cv.scores_[True], axis=0))

In [13]:
def logreg_by_lookback(reddit_df, lookback = [1], c_s = [0.1, 0.3, 1, 3, 10, 30, 100]):
    for i in lookback:
        full_df = make_dataframes(reddit_df, lookback = i, min_date = '2014-03-01')[0]
        X = full_df[['avg_weighted_compound', 'avg_weighted_positive', 'avg_weighted_negative', 'avg_weighted_neutral', 'num_posts', 'avg_positive_posts', 'avg_negative_posts']]
        y = full_df['change_percent'] > 0
        X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state = 6240)
        best_c, f1_score = cross_validate_logreg(X_train, y_train, c_s)
        print('Best f1_score = {0} for {1} day lookback with C = {2}'.format(f1_score, i, best_c))

In [14]:
logreg_by_lookback(reddit_df, lookback = [1, 3, 6, 10, 15])

Best f1_score = 0.6837936312061542 for 1 day lookback with C = 0.1
Best f1_score = 0.6847001390207956 for 3 day lookback with C = 0.1
Best f1_score = 0.6821921219983137 for 6 day lookback with C = 0.1
Best f1_score = 0.6840706556579365 for 10 day lookback with C = 0.1
Best f1_score = 0.6854336111299288 for 15 day lookback with C = 0.3


In [21]:
#test out best logistic regression model
full_df = make_dataframes(reddit_df, lookback = 15, min_date = '2014-03-01')[0]
X = full_df[['avg_weighted_compound', 'avg_weighted_positive', 'avg_weighted_negative', 'avg_weighted_neutral', 'num_posts']]
y = full_df['change_percent'] > 0
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.25, random_state = 6240)
lg = LogisticRegression(C=0.3).fit(X_train, y_train)
lg_preds = lg.predict(X_test)
f1 = f1_score(lg_preds, y_test)
precision = precision_score(lg_preds, y_test)
recall = recall_score(lg_preds, y_test)
accuracy = accuracy_score(lg_preds, y_test)
print('f1: {0}, precision: {1}, recall: {2}, accuracy: {3}'.format(f1, precision, recall, accuracy))

f1: 0.6860539163659521, precision: 1.0, recall: 0.5221324717285946, accuracy: 0.5221324717285946


In [22]:
##Baseline tests for linear regression
lookback_days = [1, 3, 6, 10, 15]
for i in lookback_days:
    full_df = make_dataframes(reddit_df, lookback = i, min_date = '2014-03-01')[0]
    X = full_df[['avg_weighted_compound', 'avg_weighted_positive', 'avg_weighted_negative', 'avg_weighted_neutral', 'num_posts']]
    y = full_df['change_percent']
    X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state = 6240)
    ln = LinearRegression(normalize=True).fit(X_train, y_train)
    ln_pred = ln.predict(X_test)
    ln_f1 = f1_score((ln_pred > 0), y_test >0)
    ln_precision = precision_score((ln_pred > 0), y_test>0)
    ln_recall  = recall_score(ln_pred>0, y_test>0)
    ln_accuracy = accuracy_score(ln_pred>0, y_test>0)
    print('{0} day lookup:'.format(i))
    print('f1: {0}, precision: {1}, recall: {2}, accuracy: {3}'.format(ln_f1, ln_precision, ln_recall, ln_accuracy))

1 day lookup:
f1: 0.6690469651194727, precision: 0.9354838709677419, recall: 0.5207353569901667, accuracy: 0.5166466105094264
3 day lookup:
f1: 0.6810666666666667, precision: 0.9762996941896025, recall: 0.5229320229320229, accuracy: 0.5200642054574639
6 day lookup:
f1: 0.6049423393739704, precision: 0.6902255639097744, recall: 0.5384164222873901, accuracy: 0.5171163914619412
10 day lookup:
f1: 0.6879014989293363, precision: 0.9801678108314263, recall: 0.5298969072164949, accuracy: 0.5292692773516351
15 day lookup:
f1: 0.6773579777113345, precision: 0.9614197530864198, recall: 0.5228703315148971, accuracy: 0.5205977382875606


In [17]:
#Random Forest
lookback_days = [1, 3, 6, 10, 15, 21, 28]
for i in lookback_days:
    full_df = make_dataframes(reddit_df, lookback = i, min_date = '2014-03-01')[0]
    X = full_df[['avg_weighted_compound', 'avg_weighted_positive', 'avg_weighted_negative', 'avg_weighted_neutral', 'num_posts']]
    y = full_df['change_percent'] > 0
    X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state = 6240)
    rf = RandomForestClassifier(n_estimators =100)
    rf.fit(X_train, y_train)
    preds = rf.predict(X_test)
    f1 = f1_score(preds, y_test)
    precision = precision_score(preds, y_test)
    recall = recall_score(preds, y_test)
    accuracy = accuracy_score(preds, y_test)
    print('{0} day lookback:'.format(i))
    print('f1: {0}, precision: {1}, recall: {2}, accuracy: {3}'.format(f1, precision, recall, accuracy))

1 day lookback:
f1: 0.595617529880478, precision: 0.6889400921658986, recall: 0.5245614035087719, accuracy: 0.5114320096269555
3 day lookback:
f1: 0.5839172505839173, precision: 0.668960244648318, recall: 0.5180580224985198, accuracy: 0.4995987158908507
6 day lookback:
f1: 0.5924439986626547, precision: 0.6661654135338346, recall: 0.5334136062612884, accuracy: 0.509061619009263
10 day lookback:
f1: 0.5748138117806364, precision: 0.6475972540045767, recall: 0.516737674984784, accuracy: 0.4929350020185709
15 day lookback:
f1: 0.5751189666893269, precision: 0.6527777777777778, recall: 0.5139732685297691, accuracy: 0.49515347334410337
21 day lookback:
f1: 0.5778546712802767, precision: 0.647788983708301, recall: 0.5215490318550906, accuracy: 0.5072697899838449
28 day lookback:
f1: 0.5765199161425576, precision: 0.6400310318076028, recall: 0.5244755244755245, accuracy: 0.5105008077544426


In [23]:
#XG Boost
lookback_days = [1, 3, 6, 10, 15]
for i in lookback_days:
    full_df = make_dataframes(reddit_df, lookback = i, min_date = '2014-03-01')[0]
    X = full_df[['avg_weighted_compound', 'avg_weighted_positive', 'avg_weighted_negative', 'avg_weighted_neutral', 'num_posts']]
    y = full_df['change_percent'] > 0
    X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state = 6240)
    xg = GradientBoostingClassifier()
    xg.fit(X_train, y_train)
    preds = xg.predict(X_test)
    f1 = f1_score(preds, y_test)
    precision = precision_score(preds, y_test)
    recall = recall_score(preds, y_test)
    accuracy = accuracy_score(preds, y_test)
    print('{0} day lookback:'.format(i))
    print('f1: {0}, precision: {1}, recall: {2}, accuracy: {3}'.format(ln_f1, ln_precision, ln_recall, ln_accuracy))

1 day lookback:
f1: 0.6773579777113345, precision: 0.9614197530864198, recall: 0.5228703315148971, accuracy: 0.5205977382875606
3 day lookback:
f1: 0.6773579777113345, precision: 0.9614197530864198, recall: 0.5228703315148971, accuracy: 0.5205977382875606
6 day lookback:
f1: 0.6773579777113345, precision: 0.9614197530864198, recall: 0.5228703315148971, accuracy: 0.5205977382875606
10 day lookback:
f1: 0.6773579777113345, precision: 0.9614197530864198, recall: 0.5228703315148971, accuracy: 0.5205977382875606
15 day lookback:
f1: 0.6773579777113345, precision: 0.9614197530864198, recall: 0.5228703315148971, accuracy: 0.5205977382875606


In [38]:
len(reddit_df.company_id.unique())

30

In [64]:
#create array of evaluation metrics by company with random forest
rf_company_results = np.zeros((30, 5))
full_df = make_dataframes(reddit_df, lookback = 28, min_date = '2014-03-01')[0]
full_df = full_df[full_df.num_posts >0]
X = full_df[['avg_weighted_compound', 'avg_weighted_positive', 'avg_weighted_negative', 'avg_weighted_neutral', 'num_posts', 'avg_positive_posts', 'avg_negative_posts']]
y = full_df['change_percent'] > 0
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state = 6240)
rf = RandomForestClassifier(n_estimators =100)
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
f1 = f1_score(preds, y_test)
precision = precision_score(preds, y_test)
recall = recall_score(preds, y_test)
accuracy = accuracy_score(preds, y_test)
dfs = make_dataframes(reddit_df, 'company', lookback = i, min_date = '2014-03-01')
print('overall scores: f1:{0}, precision: {1}, recall:{2}, accuracy{3}'.format(f1, precision, recall, accuracy))
i = 0
for df in dfs:
    c_id = df.company_id[0]
    X = df[['avg_weighted_compound', 'avg_weighted_positive', 'avg_weighted_negative', 'avg_weighted_neutral', 'num_posts', 'avg_positive_posts', 'avg_negative_posts']]
    y = df['change_percent'] > 0
    preds = rf.predict(X)
    f1 = f1_score(preds, y)
    precision = precision_score(preds, y)
    recall = recall_score(preds, y)
    accuracy = accuracy_score(preds, y)
    rf_company_results[i,] = [c_id, f1, precision, recall, accuracy]
    i+=1
        
        

overall scores: f1:0.5384615384615384, precision: 0.5535535535535535, recall:0.524170616113744, accuracy0.4960127591706539


In [77]:
#see performance of Random Forest by company
#each row shows : (company id, f1, precision, recall, accuracy)
rf_df = pd.DataFrame(rf_company_results, columns = ['company_id', 'f1', 'precision', 'recall', 'accuracy'])
companies_df[['id', 'name']].merge(rf_df, left_on='id', right_on='company_id').drop(columns ='id')

name  company_id        f1  precision  \
0               Microsoft Corporation         1.0  0.669246   0.647940   
1                          Apple Inc.         2.0  0.706320   0.698529   
2                    Amazon.com, Inc.         3.0  0.726944   0.723022   
3                      Facebook, Inc.         4.0  0.715867   0.721190   
4                        Alphabet Inc         5.0  0.678700   0.681159   
5                           eBay Inc.         6.0  0.736059   0.758621   
6                       Twitter, Inc.         7.0  0.697030   0.687500   
7                             HP Inc.         8.0  0.668501   0.920455   
8              Dell Technologies Inc.         9.0  0.771930   0.802083   
9             Uber Technologies, Inc.        10.0  0.712644   0.704545   
10                         Lyft, Inc.        11.0  0.658228   0.650000   
11                          Snap Inc.        12.0  0.773050   0.807407   
12            Spotify Technology S.A.        13.0  0.744444   0.779070   
13                          AT&T Inc.        14.0  0.747029   0.774648   
14                   T-Mobile US Inc.        15.0  0.761745   0.785467   
15                 Sprint Corporation        16.0  0.694387   0.710638   
16        Verizon Communications Inc.        17.0  0.670412   0.670412   
17               JPMorgan Chase & Co.        18.0  0.696104   0.978102   
18                     Citigroup Inc.        19.0  0.658635   0.976190   
19                    Bank of America        20.0  0.668740   0.964126   
20              Wells Fargo & Company        21.0  0.696721   0.934066   
21              PayPal Holdings, Inc.        27.0  0.744283   0.778261   
22       The American Express Company        28.0  0.689560   0.909420   
23  Capital One Financial Corporation        30.0  0.682422   0.885057   
24                      Netflix, Inc.        31.0  0.721612   0.740602   
25           DISH Network Corporation        32.0  0.657609   0.927203   
26                Comcast Corporation        33.0  0.669131   0.660584   
27                     ViacomCBS Inc.        34.0  0.650104   0.630522   
28                    Fox Corporation        35.0  0.833333   0.853659   
29                       Grubhub Inc.       121.0  0.777778   0.777778   

      recall  accuracy  
0   0.692000  0.666016  
1   0.714286  0.691406  
2   0.730909  0.705078  
3   0.710623  0.699219  
4   0.676259  0.652344  
5   0.714801  0.722656  
6   0.706827  0.701172  
7   0.524838  0.529297  
8   0.743961  0.729970  
9   0.720930  0.698795  
10  0.666667  0.674699  
11  0.741497  0.752896  
12  0.712766  0.724551  
13  0.721311  0.708984  
14  0.739414  0.722656  
15  0.678862  0.712891  
16  0.670412  0.656250  
17  0.540323  0.542969  
18  0.496970  0.501953  
19  0.511905  0.520270  
20  0.555556  0.564706  
21  0.713147  0.711944  
22  0.555310  0.558594  
23  0.555288  0.580078  
24  0.703571  0.703125  
25  0.509474  0.507812  
26  0.677903  0.650391  
27  0.670940  0.669922  
28  0.813953  0.831325  
29  0.777778  0.759036

In [52]:
#create array of performance metrics of logistic regression by company
lg_company_results = np.zeros((30, 5))
full_df = make_dataframes(reddit_df, lookback = 15, min_date = '2014-03-01')[0]
full_df = full_df[full_df.num_posts >0]
X = full_df[['avg_weighted_compound', 'avg_weighted_positive', 'avg_weighted_negative', 'avg_weighted_neutral', 'num_posts', 'avg_positive_posts', 'avg_negative_posts']]
y = full_df['change_percent'] > 0
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state = 6240)
lg = LogisticRegression()
lg.fit(X_train, y_train)
preds = lg.predict(X_test)
f1 = f1_score(preds, y_test)
precision = precision_score(preds, y_test)
recall = recall_score(preds, y_test)
accuracy = accuracy_score(preds, y_test)
dfs = make_dataframes(reddit_df, 'company', lookback = i, min_date = '2014-03-01')
i=0
for df in dfs:
    c_id = df.company_id[0]
    X = df[['avg_weighted_compound', 'avg_weighted_positive', 'avg_weighted_negative', 'avg_weighted_neutral', 'num_posts', 'avg_positive_posts', 'avg_negative_posts']]
    y = df['change_percent'] > 0
    preds = lg.predict(X)
    f1 = f1_score(preds, y)
    precision = precision_score(preds, y)
    recall = recall_score(preds, y)
    accuracy = accuracy_score(preds, y)
    lg_company_results[i,] = [c_id, f1, precision, recall, accuracy]
    i+=1

In [76]:
#see logistic regression results by company
lg_df = pd.DataFrame(lg_company_results, columns = ['company_id', 'f1', 'precision', 'recall', 'accuracy'])
companies_df[['id', 'name']].merge(lg_df, left_on='id', right_on='company_id').drop(columns ='id')

name  company_id        f1  precision  \
0               Microsoft Corporation         1.0  0.685494        1.0   
1                          Apple Inc.         2.0  0.693878        1.0   
2                    Amazon.com, Inc.         3.0  0.703797        1.0   
3                      Facebook, Inc.         4.0  0.688860        1.0   
4                        Alphabet Inc         5.0  0.700508        1.0   
5                           eBay Inc.         6.0  0.675291        1.0   
6                       Twitter, Inc.         7.0  0.666667        1.0   
7                             HP Inc.         8.0  0.680412        1.0   
8              Dell Technologies Inc.         9.0  0.725898        1.0   
9             Uber Technologies, Inc.        10.0  0.692913        1.0   
10                         Lyft, Inc.        11.0  0.650407        1.0   
11                          Snap Inc.        12.0  0.685279        1.0   
12            Spotify Technology S.A.        13.0  0.679842        1.0   
13                          AT&T Inc.        14.0  0.713568        1.0   
14                   T-Mobile US Inc.        15.0  0.721598        1.0   
15                 Sprint Corporation        16.0  0.629183        1.0   
16        Verizon Communications Inc.        17.0  0.685494        1.0   
17               JPMorgan Chase & Co.        18.0  0.697201        1.0   
18                     Citigroup Inc.        19.0  0.659686        1.0   
19                    Bank of America        20.0  0.668666        1.0   
20              Wells Fargo & Company        21.0  0.697318        1.0   
21              PayPal Holdings, Inc.        27.0  0.700152        1.0   
22       The American Express Company        28.0  0.700508        1.0   
23  Capital One Financial Corporation        30.0  0.675291        1.0   
24                      Netflix, Inc.        31.0  0.683805        1.0   
25           DISH Network Corporation        32.0  0.675291        1.0   
26                Comcast Corporation        33.0  0.697201        1.0   
27                     ViacomCBS Inc.        34.0  0.654402        1.0   
28                    Fox Corporation        35.0  0.661290        1.0   
29                       Grubhub Inc.       121.0  0.703125        1.0   

      recall  accuracy  
0   0.521484  0.521484  
1   0.531250  0.531250  
2   0.542969  0.542969  
3   0.525391  0.525391  
4   0.539062  0.539062  
5   0.509766  0.509766  
6   0.500000  0.500000  
7   0.515625  0.515625  
8   0.569733  0.569733  
9   0.530120  0.530120  
10  0.481928  0.481928  
11  0.521236  0.521236  
12  0.514970  0.514970  
13  0.554688  0.554688  
14  0.564453  0.564453  
15  0.458984  0.458984  
16  0.521484  0.521484  
17  0.535156  0.535156  
18  0.492188  0.492188  
19  0.502252  0.502252  
20  0.535294  0.535294  
21  0.538642  0.538642  
22  0.539062  0.539062  
23  0.509766  0.509766  
24  0.519531  0.519531  
25  0.509766  0.509766  
26  0.535156  0.535156  
27  0.486328  0.486328  
28  0.493976  0.493976  
29  0.542169  0.542169

In [78]:
#gradient boost
xg_company_results = np.zeros((30, 5))
full_df = make_dataframes(reddit_df, lookback = 28, min_date = '2014-03-01')[0]
full_df = full_df[full_df.num_posts >0]
X = full_df[['avg_weighted_compound', 'avg_weighted_positive', 'avg_weighted_negative', 'avg_weighted_neutral', 'num_posts', 'avg_positive_posts', 'avg_negative_posts']]
y = full_df['change_percent'] > 0
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.25, random_state = 6240)
xg = GradientBoostingClassifier(n_estimators= 100)
xg.fit(X_train, y_train)
preds = xg.predict(X_test)
f1 = f1_score(preds, y_test)
precision = precision_score(preds, y_test)
recall = recall_score(preds, y_test)
accuracy = accuracy_score(preds, y_test)
dfs = make_dataframes(reddit_df, 'company', lookback = i, min_date = '2014-03-01')
print('overall scores: f1:{0}, precision: {1}, recall:{2}, accuracy{3}'.format(f1, precision, recall, accuracy))
i=0
for df in dfs:
    c_id = df.company_id[0]
    X = df[['avg_weighted_compound', 'avg_weighted_positive', 'avg_weighted_negative', 'avg_weighted_neutral', 'num_posts', 'avg_positive_posts', 'avg_negative_posts']]
    y = df['change_percent'] > 0
    preds = xg.predict(X)
    f1 = f1_score(preds, y)
    precision = precision_score(preds, y)
    recall = recall_score(preds, y)
    accuracy = accuracy_score(preds, y)
    xg_company_results[i,] = [c_id, f1, precision, recall, accuracy]
    i+=1

overall scores: f1:0.628087986463621, precision: 0.7347585114806018, recall:0.5484633569739953, accuracy0.5327380952380952


In [79]:
#xg boost results by company
xg_df = pd.DataFrame(xg_company_results, columns = ['company_id', 'f1', 'precision', 'recall', 'accuracy'])
companies_df[['id', 'name']].merge(xg_df, left_on='id', right_on='company_id').drop(columns ='id')

name  company_id        f1  precision  \
0               Microsoft Corporation         1.0  0.663717   0.842697   
1                          Apple Inc.         2.0  0.672439   0.856618   
2                    Amazon.com, Inc.         3.0  0.691892   0.920863   
3                      Facebook, Inc.         4.0  0.677551   0.925651   
4                        Alphabet Inc         5.0  0.674253   0.858696   
5                           eBay Inc.         6.0  0.599315   0.670498   
6                       Twitter, Inc.         7.0  0.543651   0.535156   
7                             HP Inc.         8.0  0.671159   0.943182   
8              Dell Technologies Inc.         9.0  0.719682   0.942708   
9             Uber Technologies, Inc.        10.0  0.626263   0.704545   
10                         Lyft, Inc.        11.0  0.470588   0.500000   
11                          Snap Inc.        12.0  0.681199   0.925926   
12            Spotify Technology S.A.        13.0  0.698113   0.860465   
13                          AT&T Inc.        14.0  0.642968   0.732394   
14                   T-Mobile US Inc.        15.0  0.702624   0.833910   
15                 Sprint Corporation        16.0  0.483370   0.463830   
16        Verizon Communications Inc.        17.0  0.629690   0.722846   
17               JPMorgan Chase & Co.        18.0  0.696891   0.981752   
18                     Citigroup Inc.        19.0  0.659489   0.972222   
19                    Bank of America        20.0  0.661392   0.937220   
20              Wells Fargo & Company        21.0  0.696721   0.934066   
21              PayPal Holdings, Inc.        27.0  0.707741   0.973913   
22       The American Express Company        28.0  0.691257   0.916667   
23  Capital One Financial Corporation        30.0  0.669565   0.885057   
24                      Netflix, Inc.        31.0  0.674385   0.875940   
25           DISH Network Corporation        32.0  0.659401   0.927203   
26                Comcast Corporation        33.0  0.604341   0.660584   
27                     ViacomCBS Inc.        34.0  0.556263   0.526104   
28                    Fox Corporation        35.0  0.817204   0.926829   
29                       Grubhub Inc.       121.0  0.678899   0.822222   

      recall  accuracy  
0   0.547445  0.554688  
1   0.553444  0.556641  
2   0.554113  0.554688  
3   0.534335  0.537109  
4   0.555035  0.552734  
5   0.541796  0.542969  
6   0.552419  0.550781  
7   0.520921  0.523438  
8   0.581994  0.581602  
9   0.563636  0.554217  
10  0.444444  0.457831  
11  0.538793  0.548263  
12  0.587302  0.616766  
13  0.573003  0.548828  
14  0.607053  0.601562  
15  0.504630  0.544922  
16  0.557803  0.556641  
17  0.540161  0.542969  
18  0.498982  0.505859  
19  0.511002  0.518018  
20  0.555556  0.564706  
21  0.555831  0.566745  
22  0.554825  0.558594  
23  0.538462  0.554688  
24  0.548235  0.560547  
25  0.511628  0.511719  
26  0.556923  0.537109  
27  0.590090  0.591797  
28  0.730769  0.795181  
29  0.578125  0.578313

In [56]:
#linear regression
ln_company_results = np.zeros((30, 5))
full_df = make_dataframes(reddit_df, lookback = i, min_date = '2014-03-01')[0]
full_df = full_df[full_df.num_posts >0]
X = full_df[['avg_weighted_compound', 'avg_weighted_positive', 'avg_weighted_negative', 'avg_weighted_neutral', 'num_posts', 'avg_positive_posts', 'avg_negative_posts']]
y = full_df['change_percent']
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.25, random_state = 6240)
ln = LinearRegression()
ln.fit(X_train, y_train)
preds = ln.predict(X_test) > 0
f1 = f1_score(preds, y_test > 0)
precision = precision_score(preds, y_test > 0)
recall = recall_score(preds, y_test > 0)
accuracy = accuracy_score(preds, y_test > 0)
dfs = make_dataframes(reddit_df, 'company', lookback = i, min_date = '2014-03-01')
i=0
for df in dfs:
    c_id = df.company_id[0]
    X = df[['avg_weighted_compound', 'avg_weighted_positive', 'avg_weighted_negative', 'avg_weighted_neutral', 'num_posts', 'avg_positive_posts', 'avg_negative_posts']]
    y = df['change_percent']
    preds = ln.predict(X) > 0
    f1 = f1_score(preds, y>0)
    precision = precision_score(preds, y>0)
    recall = recall_score(preds, y>0)
    accuracy = accuracy_score(preds, y>0)
    ln_company_results[i,] = [c_id, f1, precision, recall, accuracy]
    i+=1

In [74]:
ln_df = pd.DataFrame(ln_company_results, columns = ['company_id', 'f1', 'precision', 'recall', 'accuracy'])
companies_df[['id', 'name']].merge(ln_df, left_on='id', right_on='company_id').drop(columns ='id')

name  company_id        f1  precision  \
0               Microsoft Corporation         1.0  0.685494   1.000000   
1                          Apple Inc.         2.0  0.693878   1.000000   
2                    Amazon.com, Inc.         3.0  0.703797   1.000000   
3                      Facebook, Inc.         4.0  0.688860   1.000000   
4                        Alphabet Inc         5.0  0.700508   1.000000   
5                           eBay Inc.         6.0  0.672727   0.992337   
6                       Twitter, Inc.         7.0  0.669281   1.000000   
7                             HP Inc.         8.0  0.672021   0.950758   
8              Dell Technologies Inc.         9.0  0.701754   0.937500   
9             Uber Technologies, Inc.        10.0  0.698413   1.000000   
10                         Lyft, Inc.        11.0  0.650407   1.000000   
11                          Snap Inc.        12.0  0.685279   1.000000   
12            Spotify Technology S.A.        13.0  0.679842   1.000000   
13                          AT&T Inc.        14.0  0.633431   0.760563   
14                   T-Mobile US Inc.        15.0  0.611801   0.681661   
15                 Sprint Corporation        16.0  0.600316   0.808511   
16        Verizon Communications Inc.        17.0  0.617600   0.722846   
17               JPMorgan Chase & Co.        18.0  0.697201   1.000000   
18                     Citigroup Inc.        19.0  0.662286   1.000000   
19                    Bank of America        20.0  0.662519   0.955157   
20              Wells Fargo & Company        21.0  0.700000   1.000000   
21              PayPal Holdings, Inc.        27.0  0.700152   1.000000   
22       The American Express Company        28.0  0.677732   0.887681   
23  Capital One Financial Corporation        30.0  0.657224   0.888889   
24                      Netflix, Inc.        31.0  0.681404   0.984962   
25           DISH Network Corporation        32.0  0.647696   0.915709   
26                Comcast Corporation        33.0  0.668547   0.864964   
27                     ViacomCBS Inc.        34.0  0.602151   0.787149   
28                    Fox Corporation        35.0  0.661290   1.000000   
29                       Grubhub Inc.       121.0  0.703125   1.000000   

      recall  accuracy  
0   0.521484  0.521484  
1   0.531250  0.531250  
2   0.542969  0.542969  
3   0.525391  0.525391  
4   0.539062  0.539062  
5   0.508841  0.507812  
6   0.502947  0.505859  
7   0.519669  0.521484  
8   0.560748  0.545994  
9   0.536585  0.542169  
10  0.481928  0.481928  
11  0.521236  0.521236  
12  0.514970  0.514970  
13  0.542714  0.511719  
14  0.554930  0.511719  
15  0.477387  0.505859  
16  0.539106  0.533203  
17  0.535156  0.535156  
18  0.495088  0.498047  
19  0.507143  0.511261  
20  0.538462  0.541176  
21  0.538642  0.538642  
22  0.548098  0.544922  
23  0.521348  0.527344  
24  0.520875  0.521484  
25  0.501048  0.492188  
26  0.544828  0.541016  
27  0.487562  0.494141  
28  0.493976  0.493976  
29  0.542169  0.542169

In [68]:
companies_df

id                       name        hq_city     hq_state  year_founded  \
0      1      Microsoft Corporation        Redmond           WA          1975   
1      2                 Apple Inc.      Cupertino           CA          1976   
2      3           Amazon.com, Inc.        Seattle           WA          1994   
3      4             Facebook, Inc.     Menlo Park           CA          2004   
4      5               Alphabet Inc  Mountain View           CA          1998   
..   ...                        ...            ...          ...           ...   
117  118  Fiat Chrysler Automobiles      Amsterdam  Netherlands          1899   
118  119      Honda Motor Co., Ltd.          Tokyo        Japan          1946   
119  120   Toyota Motor Corporation    Toyota City        Japan          1937   
120  121               Grubhub Inc.        Chicago           IL          2004   
121  122      Campbell Soup Company         Camden           NJ          1869   

     year_ipo  industry_id stock_ticker stock_exchange  
0        1986            1         MSFT         NASDAQ  
1        1980            1         AAPL         NASDAQ  
2        1997            1         AMZN         NASDAQ  
3        2012            1           FB         NASDAQ  
4        2004            1        GOOGL         NASDAQ  
..        ...          ...          ...            ...  
117      2010           17         FCAU           NYSE  
118      1977           17          HMC           NYSE  
119      1999           17           TM           NYSE  
120      2014            1         GRUB           NYSE  
121      1954            9          CPB           NYSE  

[122 rows x 9 columns]